In [1]:
from model import GPT
import tiktoken
import torch
from tqdm import tqdm

In [2]:
model = GPT.from_pretrained('gpt2-xl')

/home/mengfanxu/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading weights from pretrained gpt: gpt2-xl
forcing vocab_size=50257, block_size=1024, bias=True
number of parameters: 1555.97M


In [3]:
state_dict = torch.load("orthogonal/gpt2-xl.pt")
model.load_state_dict(state_dict)

/tmp/ipykernel_2892186/57269997.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("orthogonal/gpt2-xl.pt")


<All keys matched successfully>

In [3]:
for name, module in tqdm(model.named_modules()):
    if hasattr(module, "orthogonalize_qk_proj"):
        module.orthogonalize_qk_proj()
    if hasattr(module, "orthogonalize_vo_proj"):
        module.orthogonalize_vo_proj()

548it [14:36,  1.60s/it]


In [4]:
enc = tiktoken.get_encoding("gpt2")

In [8]:
ids = torch.LongTensor(enc.encode("American")).unsqueeze(0)
output = model.generate(idx=ids, max_new_tokens=15)
enc.decode(output[0].tolist())

'American identity. I am part of a resistance movement with my peers," he says'

In [14]:
#state_dict = model.state_dict()
#torch.save(state_dict, "orthogonal/gpt2-large.pt")

In [9]:
from copy import deepcopy
import matplotlib.pyplot as plt

In [ ]:
for layer in model.transformer.h:
    q_weight = deepcopy(layer.attn.q_proj.weight.data)
    q_bias = deepcopy(layer.attn.q_proj.bias.data).unsqueeze(-1)
    q_weight = torch.cat([q_weight, q_bias],dim=-1)
    plt.plot(q_weight.norm(p=2,dim=-1))
    plt.show()

In [ ]:
for layer in model.transformer.h:
    k_weight = deepcopy(layer.attn.k_proj.weight.data)
    k_bias = deepcopy(layer.attn.k_proj.bias.data).unsqueeze(-1)
    k_weight = torch.cat([k_weight, k_bias],dim=-1)
    plt.plot(k_weight.norm(p=2,dim=-1))
    plt.show()

In [ ]:
for layer in model.transformer.h:
    v_weight = deepcopy(layer.attn.v_proj.weight.data)
    v_bias = deepcopy(layer.attn.v_proj.bias.data).unsqueeze(-1)
    v_weight = torch.cat([v_weight, v_bias],dim=-1)
    plt.plot(v_weight.norm(p=2,dim=-1))
    plt.show()